# Optimizadores en Aprendizaje Automático

A continuación se explican los principales algoritmos de optimización usados para entrenar redes neuronales.

Como ejemplo rápido se usarán "redes" de una sola neurona (sin capas ocultas) para un caso simple de regresión lineal.

---

## 1. Gradient Descent (GD) – Descenso por Gradiente

### Idea general
Calcula el **gradiente completo** del costo usando **todo el dataset** y actualiza los parámetros una sola vez por iteración (época).

El gradiente indica la dirección de **mayor incremento** de la función de costo $J(w, b)$.  
El algoritmo se mueve en la **dirección opuesta** para minimizarla:

$$
\Delta w = -\eta \nabla_w J(w, b)
$$

donde $ \eta $ es la tasa de aprendizaje o learning rate $(lr)$.

### Función de costo
Usando el **Mean Squared Error (MSE)** como función de costo:

$$
J(w,b) = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat y_i)^2
$$

donde $n$ es el número de muestras, $y_i$ es la etiqueta real y $\hat y_i = w x_i + b$ la predicción.

### Derivadas
Las derivadas parciales del costo respecto a los parámetros son:

$$
\frac{\partial J}{\partial w} = \frac{2}{n} \sum_{i=1}^{n} (y_i - \hat y_i)(-x_i)
$$

$$
\frac{\partial J}{\partial b} = \frac{2}{n} \sum_{i=1}^{n} (y_i - \hat y_i)(-1)
$$

Al simplificar los signos:

$$
\frac{\partial J}{\partial w} = -\frac{2}{n} \sum_{i=1}^{n} (y_i - \hat y_i)x_i
$$

$$
\frac{\partial J}{\partial b} = -\frac{2}{n} \sum_{i=1}^{n} (y_i - \hat y_i)
$$

### Actualización de parámetros
En cada paso del descenso:

$$
w := w - \eta \frac{\partial J}{\partial w}
$$

$$
b := b - \eta \frac{\partial J}{\partial b}
$$

### Forma vectorizada
Para un modelo con varios parámetros, la actualización general es:

$$
\mathbf{w} := \mathbf{w} - \eta \nabla_{\mathbf{w}} J(\mathbf{w})
$$

donde $ \nabla_{\mathbf{w}} J $ es el vector gradiente de la función de costo respecto a todos los parámetros.

### Ejemplo intuitivo
- Si $J(w,b)$ es una parábola convexa, el gradiente apunta hacia la pendiente ascendente.  
- El descenso por gradiente se mueve **en la dirección contraria** al gradiente, acercándose al mínimo global.

### Consideraciones prácticas
- La cantidad de veces que se actualizan los parámetros es la misma que la cantidad de épocas de entrenamiento.
- Si $ \eta $ es **muy grande**, el algoritmo puede **oscilar o divergir**.  
- Si $ \eta $ es **muy pequeña**, la convergencia será **lenta**.   
- En funciones **no convexas**, puede quedar atrapado en **mínimos locales**.

### Características
- Usa **todas** las muestras por paso.  
- Converge de manera **suave y estable**, pero puede ser **lento** en datasets grandes.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display

# --- Dataset sintético ---
rng = np.random.default_rng(7)
n = 200
X = rng.uniform(-3, 3, size=n)
w_true, b_true = 1.8, -0.7
noise = rng.normal(0, 0.6, size=n)
Y = w_true * X + b_true + noise

# --- Función de costo MSE y su gradiente (derivada) ---
def J_mse(w, b):
    yhat = w * X + b
    e = Y - yhat
    return np.mean(e**2)

def grad_mse(w, b):
    yhat = w * X + b
    e = Y - yhat
    n = len(X)
    dw = -(2/n) * np.sum(e * X)
    db = -(2/n) * np.sum(e)
    return dw, db

# --- Parámetros para el entrenamiento ---
w0, b0 = -3.5, 3.5  # Valores iniciales de los parámetros (normalmente se eligen al azar)
lr = 0.02           # tasa de aprendizaje
epocas = 100        # Iteraciones o ciclos de entrenamiento (igual a la cantidad de actualizaciones de parámetros)

# --- Algoritmo del descenso del gradiente (GD) ---
w, b = w0, b0
path_w, path_b, path_J = [w], [b], [J_mse(w, b)]
for _ in range(epocas - 1):
    dw, db = grad_mse(w, b)
    w = w - lr * dw     # Se actualiza el parámetro w (peso)
    b = b - lr * db     # Se actualiza el parámetro b (bias)
    path_w.append(w)
    path_b.append(b)
    path_J.append(J_mse(w, b))

path_w, path_b, path_J = np.array(path_w), np.array(path_b), np.array(path_J)

# --- Hasta acá acaba la lógica del algoritmo optimizador, el resto del código es más que nada para las graficas y animaciones ---



# GRÁFICAS Y ANIMACIONES:
# --- Malla de la superficie del costo ---
pad_w, pad_b = 1.5, 1.5
wmin, wmax = min(path_w.min(), w_true) - pad_w, max(path_w.max(), w_true) + pad_w
bmin, bmax = min(path_b.min(), b_true) - pad_b, max(path_b.max(), b_true) + pad_b

W = np.linspace(wmin, wmax, 140)
B = np.linspace(bmin, bmax, 140)
WW, BB = np.meshgrid(W, B)
JJ = np.zeros_like(WW)
for i in range(WW.shape[0]):
    yhat_grid = WW[i][None, :] * X[:, None] + BB[i][None, :]
    JJ[i] = np.mean((Y[:, None] - yhat_grid) ** 2, axis=0)

# --- Figura y ejes ---
fig = plt.figure(figsize=(12, 6))
ax3d = fig.add_subplot(1, 2, 1, projection="3d")
ax2d = fig.add_subplot(1, 2, 2)

# Superficie 3D
surf = ax3d.plot_surface(WW, BB, JJ, alpha=0.82, linewidth=0, antialiased=True)
ax3d.set_xlabel("w")
ax3d.set_ylabel("b")
ax3d.set_zlabel("J(w,b)")
ax3d.view_init(elev=30, azim=45)
ax3d.set_xlim(wmin, wmax)
ax3d.set_ylim(bmin, bmax)
ax3d.set_zlim(0, JJ.max() * 1.05)

# Trayectorias
trail3d, = ax3d.plot([], [], [], lw=2, color="k")
point3d = ax3d.scatter([], [], [], s=60, c="red")
ax3d.scatter([path_w[0]], [path_b[0]], [path_J[0]], s=60, c="orange", marker="x")

# Contornos 2D (vista superior)
cs = ax2d.contourf(WW, BB, JJ, levels=30)
ax2d.contour(WW, BB, JJ, levels=15, colors="k", linewidths=0.4, alpha=0.6)
ax2d.set_xlabel("w")
ax2d.set_ylabel("b")
ax2d.set_xlim(wmin, wmax)
ax2d.set_ylim(bmin, bmax)
trail2d, = ax2d.plot([], [], lw=2, color="k")
point2d, = ax2d.plot([], [], marker="o", markersize=6, color="red")

# --- Texto dinámico ---
info_text = fig.text(
    0.12, 0.93,
    f"J(w,b) = {path_J[0]:.6f}   |   w = {path_w[0]:.4f}, b = {path_b[0]:.4f}   |   lr = {lr}",
    fontsize=12, weight="bold"
)
grad_text = fig.text(0.12, 0.90, "", fontsize=10)

# --- Funciones de animación ---
def init():
    trail3d.set_data([], [])
    trail3d.set_3d_properties([])
    point3d._offsets3d = ([], [], [])
    trail2d.set_data([], [])
    point2d.set_data([], [])
    info_text.set_text(f"J(w,b) = {path_J[0]:.6f}   |   w = {path_w[0]:.4f}, b = {path_b[0]:.4f}   |   lr = {lr}")
    grad_text.set_text("")
    return trail3d, point3d, trail2d, point2d, info_text, grad_text

def update(i):
    trail3d.set_data(path_w[:i+1], path_b[:i+1])
    trail3d.set_3d_properties(path_J[:i+1])
    point3d._offsets3d = (np.array([path_w[i]]), np.array([path_b[i]]), np.array([path_J[i]]))
    trail2d.set_data(path_w[:i+1], path_b[:i+1])
    point2d.set_data([path_w[i]], [path_b[i]])

    dw, db = grad_mse(path_w[i], path_b[i])
    info_text.set_text(
        f"J(w,b) = {path_J[i]:.6f}   |   w = {path_w[i]:.4f}, b = {path_b[i]:.4f}   |   lr = {lr}"
    )
    grad_text.set_text(f"dJ/dw = {dw:.4f}   dJ/db = {db:.4f}")
    return trail3d, point3d, trail2d, point2d, info_text, grad_text

ani = animation.FuncAnimation(fig, update, frames=len(path_w), init_func=init, interval=80, blit=False)

# --- Mostrar como HTML ---
html = ani.to_jshtml()
plt.close(fig)
display(HTML(html))


Output hidden; open in https://colab.research.google.com to view.

## 2. Stochastic Gradient Descent (SGD) – Descenso Estocástico por Gradiente

### Idea general
En lugar de usar **todo el dataset** como en el GD clásico, el SGD actualiza los parámetros **tras cada muestra individual**.  
Esto introduce cierta aleatoriedad (estocasticidad) en la trayectoria, lo que puede ayudar a escapar de mínimos locales.

Cada paso usa una sola muestra $(x_i, y_i)$ para aproximar el gradiente verdadero.  
El algoritmo se mueve en la dirección contraria al gradiente estimado:

$$
\Delta w = -\eta \nabla_w J_i(w,b)
$$

donde $J_i(w,b)$ es el costo calculado solo con la muestra $i$ y $ \eta $ es la tasa de aprendizaje o learning rate $(lr)$.

### Función de costo
El costo individual de una muestra se define como:

$$
J_i(w,b) = (y_i - \hat y_i)^2
$$

donde $\hat y_i = w x_i + b$ es la predicción del modelo lineal.

### Derivadas
Para esa única muestra se tienen las siguientes gradientes individuales:

$$
\frac{\partial J_i}{\partial w} = -2 (y_i - \hat y_i) x_i
$$

$$
\frac{\partial J_i}{\partial b} = -2 (y_i - \hat y_i)
$$

### Actualización de parámetros
En cada iteración (una muestra), se actualiza inmediatamente:

$$
w := w - \eta \frac{\partial J_i}{\partial w}
$$

$$
b := b - \eta \frac{\partial J_i}{\partial b}
$$

y luego se pasa a la siguiente muestra (posiblemente en orden aleatorio).

### Forma vectorizada
Para datasets grandes, se expresa de forma equivalente:

$$
\mathbf{w} := \mathbf{w} - \eta \nabla_{\mathbf{w}} J_i(\mathbf{w})
$$

donde el gradiente se calcula con **una sola muestra** o un subconjunto aleatorio pequeño.

### Ejemplo intuitivo
- El SGD se mueve en zigzag, porque el gradiente de cada muestra apunta en una dirección ligeramente diferente.  
- Aunque parece inestable, ese ruido puede ayudar a salir de valles planos o mínimos locales donde el GD clásico se quedaría encerrado.  
- Con suficiente número de iteraciones, converge al mismo punto que el GD, pero con una trayectoria más ruidosa.

### Consideraciones prácticas
- La cantidad de veces que se actualizan los parámetros es la cantidad de épocas de entrenamiento por la cantidad de muestras del dataset. Por lo que es un proceso más largo y lento que el GD clásico.
- Requiere **barajar aleatoriamente** las muestras en cada época para evitar sesgos.  
- Su convergencia es **más rápida por iteración**, pero **más ruidosa**.  
- Puede oscilar cerca del mínimo, por lo que a veces se aplica **momentum** o una **tasa de aprendizaje decreciente**.  
- Ideal para datasets **muy grandes**, donde calcular el gradiente completo sería costoso.

### Características
- Actualiza los parámetros **una vez por muestra**.  
- Introduce **ruido beneficioso** para explorar el espacio de soluciones.  
- Puede **no converger exactamente**, pero sí acercarse al mínimo con fluctuaciones pequeñas.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display

# --- Dataset sintético ---
rng = np.random.default_rng(7)
n = 20
X = rng.uniform(-3, 3, size=n)
w_true, b_true = 1.8, -0.7
noise = rng.normal(0, 0.6, size=n)
Y = w_true * X + b_true + noise

# --- Función de costo MSE y sus gradientes individuales ---
def J_mse(w, b):
    yhat = w * X + b
    e = Y - yhat
    return np.mean(e**2)

def grad_sample(w, b, x_i, y_i):
    yhat_i = w * x_i + b
    e_i = y_i - yhat_i
    dw = -2 * e_i * x_i
    db = -2 * e_i
    return dw, db

# --- Parámetros para el entrenamiento ---
w0, b0 = -3.5, 3.5   # Valores iniciales de los parámetros (normalmente se eligen al azar)
lr = 0.02            # tasa de aprendizaje
epocas = 20          # cada época recorre todas las muestras
steps = epocas * n   # la cantidad de actualizaciones ya no es igual a las epocas como en GD, ahora se multiplica por la cantidad de muestras

# --- Algoritmo del descenso estocástico del gradiente (SGD) ---
w, b = w0, b0
path_w, path_b, path_J = [w], [b], [J_mse(w, b)]

for epoch in range(epocas):
    indices = rng.permutation(n)
    for i in indices:
        dw, db = grad_sample(w, b, X[i], Y[i])
        w = w - lr * dw     # Se actualiza el parámetro w (peso)
        b = b - lr * db     # Se actualiza el parámetro b (bias)
        path_w.append(w)
        path_b.append(b)
        path_J.append(J_mse(w, b))

path_w, path_b, path_J = np.array(path_w), np.array(path_b), np.array(path_J)

# --- Hasta acá acaba la lógica del algoritmo optimizador, el resto del código es más que nada para las graficas y animaciones ---



# GRÁFICAS Y ANIMACIONES:
# --- Malla de la superficie del costo ---
pad_w, pad_b = 1.5, 1.5
wmin, wmax = min(path_w.min(), w_true) - pad_w, max(path_w.max(), w_true) + pad_w
bmin, bmax = min(path_b.min(), b_true) - pad_b, max(path_b.max(), b_true) + pad_b

W = np.linspace(wmin, wmax, 140)
B = np.linspace(bmin, bmax, 140)
WW, BB = np.meshgrid(W, B)
JJ = np.zeros_like(WW)
for i in range(WW.shape[0]):
    yhat_grid = WW[i][None, :] * X[:, None] + BB[i][None, :]
    JJ[i] = np.mean((Y[:, None] - yhat_grid) ** 2, axis=0)

# --- Figura y ejes ---
fig = plt.figure(figsize=(12, 6))
ax3d = fig.add_subplot(1, 2, 1, projection="3d")
ax2d = fig.add_subplot(1, 2, 2)

surf = ax3d.plot_surface(WW, BB, JJ, alpha=0.82, linewidth=0, antialiased=True)
ax3d.set_xlabel("w")
ax3d.set_ylabel("b")
ax3d.set_zlabel("J(w,b)")
ax3d.view_init(elev=30, azim=45)
ax3d.set_xlim(wmin, wmax)
ax3d.set_ylim(bmin, bmax)
ax3d.set_zlim(0, JJ.max() * 1.05)

trail3d, = ax3d.plot([], [], [], lw=2, color="k")
point3d = ax3d.scatter([], [], [], s=60, c="red")
ax3d.scatter([path_w[0]], [path_b[0]], [path_J[0]], s=60, c="orange", marker="x")

# Contornos 2D
cs = ax2d.contourf(WW, BB, JJ, levels=30)
ax2d.contour(WW, BB, JJ, levels=15, colors="k", linewidths=0.4, alpha=0.6)
ax2d.set_xlabel("w")
ax2d.set_ylabel("b")
ax2d.set_xlim(wmin, wmax)
ax2d.set_ylim(bmin, bmax)
trail2d, = ax2d.plot([], [], lw=2, color="k")
point2d, = ax2d.plot([], [], marker="o", markersize=6, color="red")

# --- Texto dinámico ---
info_text = fig.text(
    0.12, 0.93,
    f"J(w,b) = {path_J[0]:.6f}   |   w = {path_w[0]:.4f}, b = {path_b[0]:.4f}   |   lr = {lr}",
    fontsize=12, weight="bold"
)
grad_text = fig.text(0.12, 0.90, "SGD - 1 muestra por iteración", fontsize=10)

# --- Funciones de animación ---
def init():
    trail3d.set_data([], [])
    trail3d.set_3d_properties([])
    point3d._offsets3d = ([], [], [])
    trail2d.set_data([], [])
    point2d.set_data([], [])
    info_text.set_text(f"J(w,b) = {path_J[0]:.6f}   |   w = {path_w[0]:.4f}, b = {path_b[0]:.4f}   |   lr = {lr}")
    grad_text.set_text("1 muestra por iteración")
    return trail3d, point3d, trail2d, point2d, info_text, grad_text

def update(i):
    trail3d.set_data(path_w[:i+1], path_b[:i+1])
    trail3d.set_3d_properties(path_J[:i+1])
    point3d._offsets3d = (np.array([path_w[i]]), np.array([path_b[i]]), np.array([path_J[i]]))
    trail2d.set_data(path_w[:i+1], path_b[:i+1])
    point2d.set_data([path_w[i]], [path_b[i]])
    info_text.set_text(
        f"J(w,b) = {path_J[i]:.6f}   |   w = {path_w[i]:.4f}, b = {path_b[i]:.4f}   |   lr = {lr}"
    )
    return trail3d, point3d, trail2d, point2d, info_text, grad_text

ani = animation.FuncAnimation(fig, update, frames=len(path_w), init_func=init, interval=60, blit=False)

# --- Mostrar como HTML ---
html = ani.to_jshtml()
plt.close(fig)
display(HTML(html))


Output hidden; open in https://colab.research.google.com to view.

## 3. Adam Full-batch (Adaptive Moment Estimation) – Descenso Adaptativo por Gradiente

### Idea general
**Adam** es un optimizador que adapta automáticamente la magnitud de la actualización de cada parámetro, basándose en el historial de sus gradientes.  
A diferencia del GD o el SGD, que aplican la misma tasa de aprendizaje a todos los parámetros, Adam ajusta la velocidad de actualización de forma independiente para cada uno.

Adam mantiene un **promedio del gradiente reciente** y también un **promedio del cuadrado de los gradientes**.  
Esto le permite saber:
- En qué dirección se ha estado moviendo el gradiente (su tendencia).
- Cuán grandes o variables han sido esos gradientes (su escala).

Con esa información, Adam acelera el descenso en direcciones estables y lo frena en direcciones ruidosas.

### Función de costo
Como en los casos anteriores, el costo usado es el **Error Cuadrático Medio (MSE)**:

$$
J(w,b) = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat y_i)^2
$$

donde $\hat y_i = w x_i + b$.

### Derivadas
Las derivadas del costo son las mismas que en GD:

$$
\frac{\partial J}{\partial w} = -\frac{2}{n}\sum_{i=1}^{n}(y_i - \hat y_i)x_i
$$

$$
\frac{\partial J}{\partial b} = -\frac{2}{n}\sum_{i=1}^{n}(y_i - \hat y_i)
$$

### Actualización de Adam
En cada paso de entrenamiento (iteración $t$), Adam realiza tres etapas:

1. **Acumula promedios móviles de los gradientes:**

   Guarda una “memoria” del gradiente promedio $m_t$ y del cuadrado del gradiente promedio $v_t$:

   $$
   m_t = \beta_1 m_{t-1} + (1 - \beta_1) g_t
   $$
   $$
   v_t = \beta_2 v_{t-1} + (1 - \beta_2) g_t^2
   $$

   donde $g_t$ es el gradiente actual y $\beta_1, \beta_2$ son factores de suavizado (por defecto, $\beta_1 = 0.9$, $\beta_2 = 0.999$).

2. **Corrige el sesgo de los promedios:**

   Al inicio, los promedios están sesgados hacia cero. Se corrigen dividiéndolos por su factor de decaimiento:

   $$
   \hat{m}_t = \frac{m_t}{1 - \beta_1^t},
   $$

   $$
   \hat{v}_t = \frac{v_t}{1 - \beta_2^t}
   $$

3. **Actualiza los parámetros:**

   Los gradientes se escalan de forma adaptativa según la magnitud de $\hat{v}_t$:

   $$
   w_t = w_{t-1} - \eta \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon},
   $$

   $$
   b_t = b_{t-1} - \eta \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon}
   $$

   donde $\eta$ es la tasa de aprendizaje $(lr)$ y $\epsilon$ es un valor pequeño (por ejemplo $10^{-8}$) para evitar divisiones por cero.

### Forma vectorizada
Para modelos con varios parámetros:

$$
\mathbf{w}_t = \mathbf{w}_{t-1} - \eta \frac{\hat{\mathbf{m}}_t}{\sqrt{\hat{\mathbf{v}}_t} + \epsilon}
$$

### Ejemplo intuitivo
- Si los gradientes son pequeños pero consistentes, Adam los **refuerza**, avanzando más rápido.  
- Si los gradientes cambian mucho de signo o magnitud, Adam los **suaviza**, reduciendo el ruido.  
- Así, aprende más rápido que GD y con mayor estabilidad que SGD.

### Consideraciones prácticas
- Usa **valores por defecto muy estables**: $\beta_1 = 0.9$, $\beta_2 = 0.999$, $\epsilon = 10^{-8}$.  
- Puede usar una tasa de aprendizaje mayor que GD sin volverse inestable.  
- Generalmente converge más rápido y de forma más suave.  
- Es uno de los optimizadores más usados en la práctica por su rendimiento robusto.

### Características
- Calcula el gradiente usando **todo el dataset (full batch)**.  
- Ajusta el paso de aprendizaje de **forma adaptativa** por parámetro.  
- Converge **rápido y de manera estable**, incluso en funciones no convexas.  
- Reduce la sensibilidad al valor de $\eta$ y acelera el aprendizaje temprano.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display

# --- Dataset sintético ---
rng = np.random.default_rng(7)
n = 200
X = rng.uniform(-3, 3, size=n)
w_true, b_true = 1.8, -0.7
noise = rng.normal(0, 0.6, size=n)
Y = w_true * X + b_true + noise

# --- Función de costo MSE y su gradiente (derivada) ---
def J_mse(w, b):
    yhat = w * X + b
    e = Y - yhat
    return np.mean(e**2)

def grad_full(w, b):
    yhat = w * X + b
    e = Y - yhat
    n = len(X)
    dw = -(2/n) * np.sum(e * X)
    db = -(2/n) * np.sum(e)
    return dw, db

# --- Parámetros para el entrenamiento ---
w0, b0 = -3.5, 3.5  # Valores iniciales de los parámetros (normalmente se eligen al azar)
lr = 0.05           # tasa de aprendizaje
beta1, beta2 = 0.9, 0.999   # se utilizan los betas por defecto, ya que suelen dar buenos resultados
eps = 1e-8          # valor muy pequeño para epsilon
epocas = 150        # Iteraciones o ciclos de entrenamiento (igual a la cantidad de actualizaciones de parámetros)

# --- Algoritmo Adam Full-batch ---
w, b = w0, b0
m_w = m_b = 0.0
v_w = v_b = 0.0

path_w = [w]
path_b = [b]
path_J = [J_mse(w, b)]

for t in range(1, epocas):
    dw, db = grad_full(w, b)
    m_w = beta1 * m_w + (1 - beta1) * dw
    m_b = beta1 * m_b + (1 - beta1) * db
    v_w = beta2 * v_w + (1 - beta2) * (dw * dw)
    v_b = beta2 * v_b + (1 - beta2) * (db * db)
    m_w_hat = m_w / (1 - beta1**t)
    m_b_hat = m_b / (1 - beta1**t)
    v_w_hat = v_w / (1 - beta2**t)
    v_b_hat = v_b / (1 - beta2**t)
    w = w - lr * m_w_hat / (np.sqrt(v_w_hat) + eps)
    b = b - lr * m_b_hat / (np.sqrt(v_b_hat) + eps)
    path_w.append(w)
    path_b.append(b)
    path_J.append(J_mse(w, b))

path_w = np.array(path_w)
path_b = np.array(path_b)
path_J = np.array(path_J)

# --- Hasta acá acaba la lógica del algoritmo optimizador, el resto del código es más que nada para las graficas y animaciones ---



# GRÁFICAS Y ANIMACIONES:
# --- Malla de la superficie del costo ---
pad_w, pad_b = 1.5, 1.5
wmin, wmax = min(path_w.min(), w_true) - pad_w, max(path_w.max(), w_true) + pad_w
bmin, bmax = min(path_b.min(), b_true) - pad_b, max(path_b.max(), b_true) + pad_b

W = np.linspace(wmin, wmax, 140)
B = np.linspace(bmin, bmax, 140)
WW, BB = np.meshgrid(W, B)
JJ = np.zeros_like(WW)
for i in range(WW.shape[0]):
    yhat_grid = WW[i][None, :] * X[:, None] + BB[i][None, :]
    JJ[i] = np.mean((Y[:, None] - yhat_grid) ** 2, axis=0)

fig = plt.figure(figsize=(12, 6))
ax3d = fig.add_subplot(1, 2, 1, projection="3d")
ax2d = fig.add_subplot(1, 2, 2)

surf = ax3d.plot_surface(WW, BB, JJ, alpha=0.82, linewidth=0, antialiased=True)
ax3d.set_xlabel("w")
ax3d.set_ylabel("b")
ax3d.set_zlabel("J(w,b)")
ax3d.view_init(elev=30, azim=45)
ax3d.set_xlim(wmin, wmax)
ax3d.set_ylim(bmin, bmax)
ax3d.set_zlim(0, JJ.max() * 1.05)

trail3d, = ax3d.plot([], [], [], lw=2, color="k")
point3d = ax3d.scatter([], [], [], s=60, c="red")
ax3d.scatter([path_w[0]], [path_b[0]], [path_J[0]], s=60, c="orange", marker="x")

cs = ax2d.contourf(WW, BB, JJ, levels=30)
ax2d.contour(WW, BB, JJ, levels=15, colors="k", linewidths=0.4, alpha=0.6)
ax2d.set_xlabel("w")
ax2d.set_ylabel("b")
ax2d.set_xlim(wmin, wmax)
ax2d.set_ylim(bmin, bmax)
trail2d, = ax2d.plot([], [], lw=2, color="k")
point2d, = ax2d.plot([], [], marker="o", markersize=6, color="red")

info_text = fig.text(
    0.12, 0.93,
    f"J(w,b) = {path_J[0]:.6f}   |   w = {path_w[0]:.4f}, b = {path_b[0]:.4f}   |   lr = {lr}",
    fontsize=12, weight="bold"
)
grad_text = fig.text(0.12, 0.90, "", fontsize=10)

def init():
    trail3d.set_data([], [])
    trail3d.set_3d_properties([])
    point3d._offsets3d = ([], [], [])
    trail2d.set_data([], [])
    point2d.set_data([], [])
    info_text.set_text(f"J(w,b) = {path_J[0]:.6f}   |   w = {path_w[0]:.4f}, b = {path_b[0]:.4f}   |   lr = {lr}")
    grad_text.set_text("")
    return trail3d, point3d, trail2d, point2d, info_text, grad_text

def update(i):
    trail3d.set_data(path_w[:i+1], path_b[:i+1])
    trail3d.set_3d_properties(path_J[:i+1])
    point3d._offsets3d = (np.array([path_w[i]]), np.array([path_b[i]]), np.array([path_J[i]]))
    trail2d.set_data(path_w[:i+1], path_b[:i+1])
    point2d.set_data([path_w[i]], [path_b[i]])
    dw, db = grad_full(path_w[i], path_b[i])
    info_text.set_text(f"J(w,b) = {path_J[i]:.6f}   |   w = {path_w[i]:.4f}, b = {path_b[i]:.4f}   |   lr = {lr}")
    grad_text.set_text(f"dJ/dw = {dw:.4f}   dJ/db = {db:.4f}")
    return trail3d, point3d, trail2d, point2d, info_text, grad_text

ani = animation.FuncAnimation(fig, update, frames=len(path_w), init_func=init, interval=60, blit=False)

html = ani.to_jshtml()
plt.close(fig)
display(HTML(html))


Output hidden; open in https://colab.research.google.com to view.